In [58]:
import json
import csv
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy.spatial.distance

In [60]:
qa = np.genfromtxt("uid_qa.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U128'),])

In [61]:
#qa[971]

In [62]:
fe = np.genfromtxt("uid_pre_elim.txt", delimiter = ",", names = True, dtype = [('int64'), ('int64'), ('int64'), ('U256'), ('U256'),])

In [63]:
fe.dtype.names = ('uniqueID', 'bn', 'qn', 'pre_que', 'curr_elim')

#fe.dtype.names
#fe[971]

In [64]:
ftd = np.genfromtxt("face_id_descr.txt", delimiter = ";", skip_header = 1 , usecols = np.arange(0,2), dtype = [('U16'), ('U2056')])

In [65]:
ftd.dtype.names = ('img_id', 'description')
len(ftd['img_id'])
len(set(ftd['img_id']))

191

In [66]:
## first convert strings to list of rows equal to rows in fe
def mk_list(str_vector):
    des_byid = str_vector
    all_x = []
    for i in des_byid:
        j = i.split()
        #if len(j) > 0:
        all_x.append(j)

    return all_x


In [67]:
all_x = mk_list(fe['pre_que'])
len(all_x)
all_x[972]

['IMG_9078',
 'IMG_9439',
 'IMG_9273',
 'IMG_0325',
 'IMG_0491',
 'IMG_9574',
 'IMG_9409',
 'IMG_8697',
 'IMG_0716',
 'IMG_7851',
 'IMG_8834',
 'IMG_8934',
 'IMG_7866',
 'IMG_9906',
 'IMG_8667',
 'IMG_9755']

In [68]:
##check blanks
## create blank list
bl =[]
for l in range(len(all_x)):
    if len(all_x[l]) <2:
        bl.append(l)
        
        #print(l)
for l in range(len(qa['que'])):
    if len(qa['que'][l]) <2:
        bl.append(l)
list(set(bl))

[1577,
 1070,
 3119,
 3127,
 2632,
 2633,
 1611,
 2132,
 2133,
 86,
 605,
 2141,
 2142,
 2143,
 2144,
 113,
 1148,
 128,
 2185,
 2186,
 1197,
 688,
 690,
 698,
 195,
 198,
 1735,
 1739,
 2271,
 3295,
 2786,
 2277,
 2286,
 2288,
 2293,
 2298,
 2307,
 2310,
 1806,
 2325,
 2849,
 2851,
 2856,
 306,
 307,
 308,
 309,
 310,
 2368,
 2386,
 853,
 1371,
 1381,
 360,
 361,
 367,
 374,
 375,
 380,
 2428,
 391,
 2446,
 1428,
 1429,
 1431,
 1432,
 1963,
 1463,
 952,
 953,
 1464,
 1465,
 1466,
 1470,
 1471,
 1472,
 1473,
 1474,
 1988,
 1476,
 1477,
 1478,
 1993,
 971,
 2507,
 1483,
 977,
 474]

In [69]:
## remove blanks from all 
        
     

In [70]:
## Now for each row in the list above, find description and add it
all_i_txt = []

for rown in range(len(all_x)):
    #if rown not in bl:
        #print(all_x.index)
    int_i_txt = []
    for img in all_x[rown]:
        if img in ftd['img_id']:
            d = str(ftd['description'][list(ftd['img_id']).index(img)])
            int_i_txt.append(str(d))
    all_i_txt.append(int_i_txt)
        
            
            

In [71]:
#print check
#print(type(all_i_txt[0]))
print(len(all_i_txt))
print(all_i_txt[971])

3314
[]


In [72]:
## add question and answer to each data point
for it in all_i_txt:
    it.extend([qa['que'][all_i_txt.index(it)], qa['ans'][all_i_txt.index(it)]])
    
    

In [73]:
## check addition of above objects
for i in range(len(all_i_txt)):
    print(all_i_txt[i][-2:])

[' Does the person have glasses?', ' No']
[' Does the person have writing on their shirt?', ' No']
[' Is the person a boy?', ' No']
[' Does the person have glasses?', ' Yes']
[' Does the person have writing on their shirt?', ' No']
[' Does the person have blonde hair?', ' No']
[' Does the person have something on their head?', ' No']
[' Does the person have writing on their shirt?', ' No']
[' Is the person a boy?', ' No']
[' Does the person have writing on their shirt?', ' No']
[' Does she have blond hair?', ' No']
[' does she have blond hair?', ' Yes']
[' Is she black?', ' No']
[' is she asian?', ' No']
[' Is she black?', ' Yes']
[' Is she blond?', ' Yes']
[' Does he have a beard?', ' Yes']
[' Is his hair flat?', ' No']
[' is his hair flat?', ' No']
[' does he have glasses?', ' Yes']
[' old?', ' Yes']
[' old?', ' Yes']
[' short hair?', ' No']
[' short hair?', ' Yes']
[' blond?', ' Yes']
[' beard?', ' No']
[' tall?', ' Yes']
[' old?', ' Yes']
[' glasses?', ' Yes']
[' black?', ' Yes']
[

In [74]:
#for i in set(bl):
#    print(all_i_txt[i][-2:])

In [75]:
#print(all_i_txt[0])
##basic model
cv = CountVectorizer()

In [76]:
## all similarities
all_sim = []
for l in range(len(all_i_txt)):
    H = cv.fit_transform(all_i_txt[l])
    sim = cosine_similarity(H)
    all_sim.append(sim)

In [116]:
#similarities[-2] meaning taking only the array for question
all_sim_q = []
for i in range(len(all_sim)):
    all_sim_q.append(all_sim[i][-2][:-2])
print(len(all_sim_q))
#print(all_sim_q[971])
#print(len(all_sim))

3314


In [117]:
all_cheat = mk_list(fe['curr_elim'])

In [118]:
cheat_code = zip(all_sim_q, all_cheat)

In [119]:
## Sort all the above similarities with argsort which gives indices
## or we should not take top5 and make a threshold above which we will include it in the list of output?
sorted_ind = []
for si, ln in cheat_code:
    y_top = si.argsort()[::-1][:len(ln)]
    sorted_ind.append(y_top)
#sorted_ind[971]


The problem is below. I still have cases where there is only one image id. 
Even though I have made a blank list above. Where I have found out cases where there is no image id before asking the question or one image id before asking the que. Or where que is blank. 

In [120]:
top_img = zip(all_x, sorted_ind)


In [121]:
## convert to img_id from index to match with actual image ids selcted by participant
def mk_imgid(arr):
    y_pred = []
    for i,j in arr:
        yp = []
        for s in j:
            yp.append(i[s])
        y_pred.append(yp)
    return y_pred
    
    

In [122]:
pred_img = mk_imgid(top_img)

In [123]:
#print(all_cheat[0:3])
#v = mk_imgid(top_img)
#print(w)

In [135]:
acc_in = zip(all_cheat, pred_img)

In [136]:
acc_all = []
for a,p in acc_in:
    #print(a,p)
    if len(a) == 0:
        acc_i = 0
    else:
        acc_i = len(set(a).intersection(set(p)))/len(set(a))
    acc_all.append(acc_i)

In [137]:
#len(pred_img)
print(acc_all)

[0.6666666666666666, 0.9166666666666666, 0.625, 0.6, 0.0, 1.0, 0.5, 0.9230769230769231, 0.5, 0.8571428571428571, 0, 0.0, 0, 0.0, 0.8333333333333334, 0.0, 0.125, 0.6666666666666666, 0, 0.75, 0, 0, 0, 0.9285714285714286, 1.0, 0.75, 0.6666666666666666, 0.5, 0, 0, 0.0, 0.5555555555555556, 0.8333333333333334, 0.75, 0.9090909090909091, 0.375, 0.8571428571428571, 0.6666666666666666, 1.0, 0.375, 0.4, 0.5, 0.6666666666666666, 0.0, 1.0, 0.0, 0.9090909090909091, 0, 1.0, 0.4444444444444444, 0.8333333333333334, 0, 0.42857142857142855, 0.6666666666666666, 0.5, 0.25, 0.0, 0.5, 0.0, 1.0, 0, 1.0, 0.5, 0.3333333333333333, 0.0, 1.0, 0.6666666666666666, 0.7142857142857143, 0.0, 0.0, 0.7272727272727273, 0.6666666666666666, 0.0, 0.2857142857142857, 0.3333333333333333, 1.0, 0.5, 0.4, 1.0, 0.0, 0.5, 0.5, 0.8571428571428571, 0.14285714285714285, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.25, 0, 0, 0, 0.7, 1.0, 0.0, 0, 0.42857142857142855, 0.0, 0.0, 0.75, 0.7692307692307693, 1.0, 0.6363636363636364, 1.0, 0.5, 0.42857142

In [138]:
np.mean(np.array(acc_all))

0.46677424324150335